In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import math
from scipy.integrate import solve_ivp
from scipy import integrate
from matplotlib.pyplot import figure
import ordpy
import array
import statistics

def JJ (t, y, xi_0, T_1_, xi_P, A_P, omega_P, A_0, omega_T_, alpha, beta, c, b_0, a, T_2_, g):
    x1, y1, x2, y2, z = y
    dx1 = x1*(1-xi_0*math.e**(-1/T_1_)-xi_P)-y1-1/3*x1**3+A_P*math.cos(omega_P*t)+A_0*math.e**(-1/T_1_)*math.cos(omega_T_*t)-alpha*math.sin(z)-beta*(x1-x2)
    dy1 = c*(-b_0*math.e**(1/T_1_)*y1+a+x1)
    dx2 = x2*(1-xi_0*math.e**(-1/T_2_)-xi_P)-y2-1/3*x2**3+A_P*math.cos(omega_P*t)+A_0*math.e**(-1/T_2_)*math.cos(omega_T_*t)-alpha*math.sin(z)-beta*(x1-x2)
    dy2 = c*(-b_0*math.e**(1/T_2_)*y2+a+x2)
    dz = g*(x1-x2)
    dydt = [dx1, dy1, dx2, dy2, dz]
    return dydt


xi_0 = 0.175
T_1_ = 5
xi_P = 0.175
A_P = 0.01
omega_P = 0.7
A_0 = 0.5
omega_T_ = 0.7
alpha = 0.002
beta = 0.037
c = 0.1
b_0 = 0.6
a = 0.7
T_2_ = 5
g = 0.1
probability_difference_list_for_truncated_ts=[]

# complete time series graph
tStart = 0
tEnd = 8000

X0 = [0.2, 0.1, 0.02, 0.01, 0.001]
solution = integrate.solve_ivp(JJ, [tStart, tEnd], X0, method='RK45', t_eval=np.linspace(tStart,tEnd,100000), args=(xi_0, T_1_, xi_P, A_P, omega_P, A_0, omega_T_, alpha, beta, c, b_0, a, T_2_, g))

t_sol = solution.t
x1_sol = solution.y[0]
y1_sol = solution.y[1]
x2_sol = solution.y[2]
y2_sol = solution.y[3]
z_sol = solution.y[4]

# error function
error_func = []
i = 0
while i<len(t_sol):
    error_func.append(math.sqrt((x1_sol[i]-x2_sol[i])**2+(y1_sol[i]-y2_sol[i])**2))
    i += 1

# P(pattern0123) - P(pattern3210)
ordinal_distribution = ordpy.ordinal_distribution(error_func, dx=4, dy=1, taux=1, tauy=1, return_missing=False, tie_precision=None)
probability_difference=ordinal_distribution[1][0]-ordinal_distribution[1][len(ordinal_distribution[1])-1]
print('probability difference of time interval {0} to {1} = {2}'.format(tStart, tEnd, probability_difference))

figure(figsize=(120, 8))
plt.plot(t_sol, error_func)
plt.xlabel('t', fontsize=60)
plt.ylabel(''r'$\theta$', fontsize=60)
plt.title(''r'$\alpha={0}$, 'r'$\beta={1}$, t={2}-{3}'.format(alpha, beta, tStart, tEnd), fontsize=60)
plt.rcParams['xtick.labelsize'] = 60
plt.rcParams['ytick.labelsize'] = 60
plt.show()

i=0
t_delta=500
i_min=0
i_max=int(t_delta*(len(t_sol)/tEnd))
while i_min < len(t_sol):
    error_func.clear()
    while i_min <= i < i_max:
        error_func.append(math.sqrt((x1_sol[i]-x2_sol[i])**2+(y1_sol[i]-y2_sol[i])**2))
        i += 1
    ordinal_distribution = ordpy.ordinal_distribution(error_func, dx=4, dy=1, taux=1, tauy=1, return_missing=False, tie_precision=None)
    probability_difference=ordinal_distribution[1][0]-ordinal_distribution[1][len(ordinal_distribution[1])-1]
    print('time interval = {0} - {1}, probability difference = {2}'.format(i_min/(len(t_sol)/tEnd), i_max/(len(t_sol)/tEnd), probability_difference))
    probability_difference_list_for_truncated_ts.append(probability_difference)
    i_min += int(t_delta*(len(t_sol)/tEnd))
    i_max += int(t_delta*(len(t_sol)/tEnd))

print(probability_difference_list_for_truncated_ts)
print('mean of probability difference list = {0}'.format(sum(probability_difference_list_for_truncated_ts)/(tEnd/t_delta)))
print('standard deviation of probability difference list = {0}'.format(statistics.stdev(probability_difference_list_for_truncated_ts)))